# ML Pipeline Preparation

### 1. Importing libraries and loading data from data base

In [1]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\x\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import datetime


from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords 


from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split
from sklearn.multioutput import MultiOutputClassifier
"""
from sklearn.ensemble import RandomForestClassifier
Random Forest is too time consuming for typical single machine. If you want too see the results
for the Random Forest Classifier please go back to the very first version of this project
"""
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import classification_report

### Optional display settings

In [3]:
plt.style.use('seaborn-dark')
pd.set_option("display.max_rows", 1000)

In [4]:
engine = create_engine('sqlite:///Pipeline_Project.db')
df = pd.read_sql_table('Messages', engine)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4445 entries, 0 to 4444
Data columns (total 37 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      4445 non-null   int64  
 1   message                 4445 non-null   object 
 2   related                 4445 non-null   float64
 3   request                 4445 non-null   float64
 4   offer                   4445 non-null   float64
 5   aid_related             4445 non-null   float64
 6   medical_help            4445 non-null   float64
 7   medical_products        4445 non-null   float64
 8   search_and_rescue       4445 non-null   float64
 9   security                4445 non-null   float64
 10  military                4445 non-null   float64
 11  water                   4445 non-null   float64
 12  food                    4445 non-null   float64
 13  shelter                 4445 non-null   float64
 14  clothing                4445 non-null   

### General division into labels and features

In [5]:
X = df.iloc[:, 1]
Y = df.iloc[:, 2:]

### 2. Key functions

#### Tokenizer for processing the text data

In [6]:
def tokenize(text):
    #Normalization - lowercase  - no punctuation removal - the nltk.tokenize should interpret them by itself
    text = text.lower()
    ######text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    #Toeknization
    ######words = text.split()
    words = word_tokenize(text)
    #Stop words removal
    words = [w for w in words if w not in stopwords.words("english")]
    
    #Named Entities??
    # Perhaps
    
    #Lemmatization and Stemming
    lemmed = [WordNetLemmatizer().lemmatize(w) for w in words]
    stemmed  = [PorterStemmer().stem(w) for w in lemmed]
    
    return stemmed

#### Since I want to test out many models, it seems reasonable to write a function which will make pipeline with keyword arguments

In [7]:
def make_sklearn_pipeline(Pipeline = Pipeline, memory = None, verbose = False, **kwargs):
    """
    Firt argument is sklearn Pipepline class. It should not be changed.
    
    Definition of pipeline steps happens EXPLICITLY within the instantiation!
    Definition of typical pipeline looks like following:
    example_pipeline = make_sklearn_pipeline(steps = [('name#1',function#1),
                                                      ('name#2',function#2),
                                                      ('name#3',function#3),
                                                      ('name#4',function#4)....], verbose = ...,
                                                      memory = ...)
    
    It is not required to give values for ´verbose´ and ´memory´. They have default values as False
    and None respectively. For more information visit: 
    ´https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html´
    """
    pipeline = Pipeline(steps = kwargs['steps'], memory = memory, verbose = verbose)
    
    return pipeline

#### Function transforming the classification report into data frame with recall, precision and accuracy only 

In [8]:
def df_from_sklearn_cl_reports(cl_reports):

    data_frame = pd.DataFrame()

    for feature in list(cl_reports.keys()):
        l = cl_reports[feature].split(' ')
        l = [x for x in l if x !='']
        l = [x for x in l if '\n' not in x]
        l = l[:l.index('accuracy')]
        columns = l[:3]
        l = [x for x in l if x not in columns]
        labels = []
        precisions = []
        recalls = []
        f1_scores = []
    
        for i, element in enumerate(l):
            if i == 0 or i % 4 == 0:
                labels.append(element)
                precisions.append(l[i+1])
                recalls.append(l[i+2])
                f1_scores.append(l[i+3])
            
        comunicates = [feature]*len(labels)
    
        if data_frame.shape[0] == 0:      
            data_frame['communicate'] = comunicates
            data_frame['label'] = labels
            data_frame['precisions'] = precisions
            data_frame['recalls'] = recalls
            data_frame['f1_scores'] = f1_scores
        
        else:
            auxiliary_df = pd.DataFrame()
            auxiliary_df['communicate'] = comunicates
            auxiliary_df['label'] = labels
            auxiliary_df['precisions'] = precisions
            auxiliary_df['recalls'] = recalls
            auxiliary_df['f1_scores'] = f1_scores 
        
            data_frame = pd.concat([data_frame, auxiliary_df])
        
            del auxiliary_df
    
    data_frame.set_index(["communicate"], inplace = True)
    
    #Chaning the data types
    data_frame.label = data_frame.label.astype("float")
    data_frame.label = data_frame.label.astype("int")
    data_frame.precisions = data_frame.precisions.astype("float")
    data_frame.recalls = data_frame.recalls.astype("float")
    data_frame.f1_scores = data_frame.f1_scores.astype("float")
    
    return data_frame    

### 3. Building a machine learning pipeline

#### 3.1 AdaBoost Classifier

##### Definition and fitting

In [9]:
adaboost_with_transformer = make_sklearn_pipeline(verbose = True, steps = [('vect', CountVectorizer(tokenizer = tokenize)),
                           ('tfidf', TfidfTransformer()),                                              
                           ('clf', MultiOutputClassifier(estimator = AdaBoostClassifier()))])

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size = 0.3)
adaboost_with_transformer.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=  35.5s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=   9.9s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001AC69695048>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
         verbose=True)

##### Predicting and classification report

In [11]:
y_pred = adaboost_with_transformer.predict(X_test)
y_pred = pd.DataFrame(y_pred, columns = list(Y.columns))
y_test = y_test.reset_index(drop = True)
for i, var in enumerate(Y):
    print(var)
    print(classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]))

related
              precision    recall  f1-score   support

         0.0       0.46      0.19      0.27       220
         1.0       0.86      0.96      0.90      1114

    accuracy                           0.83      1334
   macro avg       0.66      0.57      0.58      1334
weighted avg       0.79      0.83      0.80      1334

request
              precision    recall  f1-score   support

         0.0       0.88      0.93      0.91      1012
         1.0       0.73      0.61      0.67       322

    accuracy                           0.85      1334
   macro avg       0.81      0.77      0.79      1334
weighted avg       0.85      0.85      0.85      1334

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1328
         1.0       0.00      0.00      0.00         6

    accuracy                           0.99      1334
   macro avg       0.50      0.50      0.50      1334
weighted avg       0.99      0.99      0.99      133

##### Providing a data frame for easier results assessment

In [12]:
classification_reports = {}

In [13]:
for i, var in enumerate(Y):
    classification_reports[var] = (classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]));

In [14]:
adaboost_with_transformer_report_df = df_from_sklearn_cl_reports(classification_reports)

In [15]:
adaboost_with_transformer_report_df.loc[
    adaboost_with_transformer_report_df.label == 1].groupby(
    ["communicate"])["precisions"].mean().sort_values(ascending = False)

communicate
earthquake                0.94
clothing                  0.90
weather_related           0.88
related                   0.86
food                      0.84
floods                    0.80
water                     0.78
aid_related               0.78
storm                     0.78
shelter                   0.77
request                   0.73
direct_report             0.62
medical_products          0.61
buildings                 0.59
other_aid                 0.57
electricity               0.55
search_and_rescue         0.55
death                     0.53
transport                 0.52
hospitals                 0.50
medical_help              0.49
other_weather             0.48
military                  0.46
cold                      0.46
refugees                  0.44
money                     0.40
infrastructure_related    0.22
other_infrastructure      0.17
fire                      0.17
missing_people            0.14
offer                     0.00
security                  0

In [16]:
adaboost_with_transformer_report_df.loc[
    adaboost_with_transformer_report_df.label == 1].groupby(
    ["communicate"])["recalls"].mean().sort_values(ascending = False)

communicate
related                   0.96
earthquake                0.83
food                      0.77
water                     0.72
weather_related           0.72
aid_related               0.72
request                   0.61
shelter                   0.57
floods                    0.56
clothing                  0.51
direct_report             0.51
storm                     0.39
buildings                 0.37
military                  0.33
medical_products          0.30
death                     0.29
medical_help              0.28
other_aid                 0.26
electricity               0.24
money                     0.22
search_and_rescue         0.19
cold                      0.18
other_weather             0.16
transport                 0.16
refugees                  0.13
fire                      0.12
missing_people            0.08
hospitals                 0.07
infrastructure_related    0.07
other_infrastructure      0.06
offer                     0.00
security                  0

##### Trying to ehence the model's performance via GridSearch

##### Decision tree has been chosen for the base estimator 

In [17]:
dt_depth_2 = DecisionTreeClassifier(max_depth=2, min_samples_leaf=1)
dt_depth_9 = DecisionTreeClassifier(max_depth=9, min_samples_leaf=1)

In [18]:
parameters = {
    'clf__estimator__base_estimator': [dt_depth_2,dt_depth_9],
    'clf__estimator__n_estimators': [50,100]    
}

In [19]:
adaboost_transf_optimized = GridSearchCV(adaboost_with_transformer, param_grid = parameters)

In [20]:
#Attention! This cell may take a bit time to perform (around 15 minutes on typical PC)
adaboost_transf_optimized.fit(X_train, y_train)

[Pipeline] .............. (step 1 of 3) Processing vect, total=  27.5s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  14.3s
[Pipeline] .............. (step 1 of 3) Processing vect, total=  27.4s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  14.3s
[Pipeline] .............. (step 1 of 3) Processing vect, total=  28.3s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  14.5s
[Pipeline] .............. (step 1 of 3) Processing vect, total=  27.6s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipeline] ............... (step 3 of 3) Processing clf, total=  14.3s
[Pipeline] .............. (step 1 of 3) Processing vect, total=  26.3s
[Pipeline] ............. (step 2 of 3) Processing tfidf, total=   0.0s
[Pipel

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001AC69695048>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))],
                                verbose=True),
             param_grid={'clf__estimator__base_estimator': [DecisionTreeClassifier(max_depth=2),
                                                            DecisionTreeClassifier(max_depth=9)],
                         'clf__estimator__n_estimators': [50, 100]})

In [21]:
y_pred_opt = adaboost_transf_optimized.predict(X_test)
y_pred_opt = pd.DataFrame(y_pred_opt, columns = list(Y.columns))
y_test = y_test.reset_index(drop = True)
for i, var in enumerate(Y):
    print(var)
    print(classification_report(y_test.iloc[:,i], y_pred_opt.iloc[:,i]))

related
              precision    recall  f1-score   support

         0.0       0.44      0.29      0.35       220
         1.0       0.87      0.93      0.90      1114

    accuracy                           0.82      1334
   macro avg       0.65      0.61      0.62      1334
weighted avg       0.80      0.82      0.81      1334

request
              precision    recall  f1-score   support

         0.0       0.88      0.91      0.89      1012
         1.0       0.68      0.59      0.63       322

    accuracy                           0.84      1334
   macro avg       0.78      0.75      0.76      1334
weighted avg       0.83      0.84      0.83      1334

offer
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      1328
         1.0       0.00      0.00      0.00         6

    accuracy                           1.00      1334
   macro avg       0.50      0.50      0.50      1334
weighted avg       0.99      1.00      0.99      133

C:\Users\x\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\x\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\x\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\x\Anaconda3\lib\site-packages\sklearn\metrics\_classificat

In [22]:
classification_reports = {}

In [23]:
for i, var in enumerate(Y):
    classification_reports[var] = (classification_report(y_test.iloc[:,i], y_pred.iloc[:,i]));

In [24]:
adaboost_optimized_report_df = df_from_sklearn_cl_reports(classification_reports)

In [25]:
adaboost_optimized_report_df.loc[
    adaboost_optimized_report_df.label == 1].groupby(
    ["communicate"])["precisions"].mean().sort_values(ascending = False)

communicate
earthquake                0.94
clothing                  0.90
weather_related           0.88
related                   0.86
food                      0.84
floods                    0.80
water                     0.78
aid_related               0.78
storm                     0.78
shelter                   0.77
request                   0.73
direct_report             0.62
medical_products          0.61
buildings                 0.59
other_aid                 0.57
electricity               0.55
search_and_rescue         0.55
death                     0.53
transport                 0.52
hospitals                 0.50
medical_help              0.49
other_weather             0.48
military                  0.46
cold                      0.46
refugees                  0.44
money                     0.40
infrastructure_related    0.22
other_infrastructure      0.17
fire                      0.17
missing_people            0.14
offer                     0.00
security                  0

In [26]:
adaboost_optimized_report_df.loc[
    adaboost_optimized_report_df.label == 1].groupby(
    ["communicate"])["recalls"].mean().sort_values(ascending = False)

communicate
related                   0.96
earthquake                0.83
food                      0.77
water                     0.72
weather_related           0.72
aid_related               0.72
request                   0.61
shelter                   0.57
floods                    0.56
clothing                  0.51
direct_report             0.51
storm                     0.39
buildings                 0.37
military                  0.33
medical_products          0.30
death                     0.29
medical_help              0.28
other_aid                 0.26
electricity               0.24
money                     0.22
search_and_rescue         0.19
cold                      0.18
other_weather             0.16
transport                 0.16
refugees                  0.13
fire                      0.12
missing_people            0.08
hospitals                 0.07
infrastructure_related    0.07
other_infrastructure      0.06
offer                     0.00
security                  0

>Undersampling still gives poor results - escpecially for recall which is very important in disaster response system. Undersampling may not necesserilly work good in multioutput problems, because of possible noise beeing added to the data set.
Moreover, if we really want to balance all of the classes via undersampling we will end up with very small data set (some imbalanced classes are much less numerous than the better balanced ones). Hence the next iterration will be conducted with **´class weights´** adjustments and without any reduction of the data set size.